# process Anagada Passage Sections

In this notebook we are going to start creating some `science` products from our RU29 data!

So far we have:
1. identified all the start and end times of the crossings (`01_get_anagada_sections`)
1. gridded and split the glider data into sections, and saved them as individual netcdf files (`02_create_anagada_netcdfs`)

Next let's start to apply some of our physical Oceanographic knowledge to "derive" important quantities from the data. 

We will start with calculating the geostrophic velocity using the thermal wind equations:

$$
\frac{\partial v}{\partial z} = \frac{-g}{\rho f}\frac{\partial \rho}{\partial x} \\
\frac{\partial u}{\partial z} = \frac{g}{ \rho f}\frac{\partial \rho}{\partial y}
$$

How do you actually calculate those equations from real data?

Talk through this, and ask Nick to draw on his ipad

remember, from our glider data we can measure things on the right hand side:

$$
\frac{-g}{\rho f}\frac{\partial \rho}{\partial x} \\
\frac{g}{ \rho f}\frac{\partial \rho}{\partial y}
$$

at least we can measure $\frac{\partial \rho}{\partial x}$ in the direction the glider is flying (not in the cross flight direction), so we can calculate the left hand side, or how the velocity changes with depth: 

$$
\frac{\partial v}{\partial z} 
$$

# comparison with earlier results:

[Bill Johns et al in 2002](https://doi.org/10.1016/S0967-0637(01)00041-3) give an overview of the inflow of water to the carrabean. 

We can compare to this paper!

![](./johns_2002.png)

In [8]:
!pwd

/Users/beaird/Google Drive/Documents/IOO_masters/courses_complete/2020/FM2020/glider_tracking


In [6]:
import xarray as xr
import matplotlib.pyplot as plt
import seawater as sw
import numpy as np
from scipy import integrate

from scipy import interpolate

import cartopy.crs as ccrs

In [7]:
fname = './processed_sections/section1.nc'

ds = xr.open_dataset(fname)


ds

<xarray.Dataset>
Dimensions:                (date: 29, depth: 199)
Coordinates:
  * date                   (date) datetime64[ns] 2020-10-01T00:26:24.18230579...
  * depth                  (depth) float64 2.5 7.5 12.5 ... 982.5 987.5 992.5
    lat                    (date) float64 ...
    lon                    (date) float64 ...
    distance               (date) float64 ...
Data variables:
    u                      (date) float64 ...
    v                      (date) float64 ...
    potential_temperature  (depth, date) float64 ...
    salinity               (depth, date) float64 ...
    cdom                   (depth, date) float64 ...
    chlorophyll_a          (depth, date) float64 ...
    beta_700nm             (depth, date) float64 ...

# make a map

In [ ]:
# grab bathymetry data for a map

lat_lims = [16, 18.9]
lon_lims = [-66, -61]

# grab some bathymetry

url = 'http://geoport.whoi.edu/thredds/dodsC/bathy/etopo1_bed_g2'

etopo1 = xr.open_dataset(url).sel(lat = slice(lat_lims[0], lat_lims[1]), lon = slice(lon_lims[0], lon_lims[1] ))



# how big is DAC?

# get $\rho$

In [ ]:
# calculate potential density



In [ ]:
# look at the density anomaly on depths



# get $\frac{\partial \rho}{\partial y}$


What are potential issues with this?

How might we deal with them? - a potential GDA project

# get $\frac{\partial u}{\partial z}$

ok, now we have all the components of the right hand side to give us how the velocity changes with depth :

$$
\frac{g}{\rho f}\frac{\partial \rho}{\partial y}
$$

In [ ]:
# # # calculate the thermal wind


# Integrate vertically to get $u$

So far we have $\frac{\partial u}{\partial z}$, or how velocity changes with depth. But what  we would really like to know is the total velocity $u$. 

We can integrate the shear to get this!

$$
u = \int \frac{du}{dz}dz
$$


or actually each little step in $v$ looks like this (just re arrainging the equation):

$$
\Delta u = \frac{\Delta u}{\Delta z} * \Delta z
$$

if you sub in the thermal wind equation it looks like this:

$$
\Delta u = \frac{-g}{\rho f}\frac{\Delta \rho}{\Delta y} * \Delta z
$$

and if we want the total $u$ we can sum up all the little $\Delta u$'s from the water column to get a profile of $u$

This actually just gives us a `relative geostrophic velocity` something we will have to reference to get the absolute velocity

In [ ]:

# we can take the integral by cumulativly summing the thermal wind
# we must multiply by the thickness of the depth bin



# adjust the relative velocity

So far we assume that the velocity starts at `0` at 1000m and get `ds.V_rel`. This probably isn't true. 

Luckily we know what the depth-average (0-1000m average) velocity is. We can just set the depth average of `ds.V_rel` equal to that measurement!

There are a couple of steps:
* remember we actually only get the component of velocity in the direction perpendicular to the glider flight path, so we need just that component of the DAC to use as a reference. We do this by first identifying the direction the glider is going, then taking the projection of the DAC in the perpendicular direction
* we also don't have DAC everywhere right now, so we will probably need to do some interpolation

### note: 
The glider doesn't fly in a totally straight line. We could 'map' the glider data onto an actual straight line to make the calculations easier. We can think about this later- could be a GDA project


# add the reference velocity

$$
V_{abs}(x,z) = V_{rel}(x,z) + V_{ref}(x)
$$

The reference, $V_{ref}$, is calculated from the DAC ($V_{dac}$) such that:

$$
V_{ref}(x) = V_{dac} - \frac{1}{h}\int_h V_{g}(x,z)dz
$$